# **Data Collection**

## Objectives

* Fetch data from Kaggle and prepare it for further processes

## Inputs

* Kaggle JSOn file - authentication token 

## Outputs

* Generate Dataset: inputs/datasets/housing-prices-data 

## Additional Comments

* No comments 


---

# Change working directory

In [1]:
import numpy
import os

In [2]:
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-housing-issues/jupyter_notebooks'

Change the working directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-housing-issues'

---

# Load Data

In [5]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/house_price_records_cat.csv"))
df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856.0,854.0,3.0,No,706.0,GLQ,150.0,548.0,RFn,2003,...,8450.0,65.0,196.0,61.0,5,7,856.0,2003,2003,208500
1,1262.0,0.0,3.0,Gd,978.0,ALQ,284.0,460.0,RFn,1976,...,9600.0,80.0,0.0,0.0,8,6,1262.0,1976,1976,181500
2,920.0,866.0,3.0,Mn,486.0,GLQ,434.0,608.0,RFn,2001,...,11250.0,68.0,162.0,42.0,5,7,920.0,2001,2002,223500
3,796.0,566.0,1.0,No,732.0,GLQ,64.0,480.0,Unf,1993,...,14115.0,85.0,0.0,30.0,5,5,796.0,1993,1995,143000
4,1694.0,0.0,3.0,Av,1369.0,GLQ,317.0,636.0,RFn,2004,...,10084.0,75.0,186.0,57.0,5,8,1686.0,2004,2005,307000


## Data Exploration

*Checking variable type and distribution, missing levels and what these variables mean in a business context*

In [6]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

/tmp/ipykernel_5873/962869596.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Correlation Study

*Variables are already int or float therefore there is not need to user OneHotEncoder*

In [7]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=df.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head()

(797, 37)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtFinSF1,BsmtUnfSF,GarageArea,GarageYrBlt,GrLivArea,LotArea,LotFrontage,...,BsmtFinType1_None,BsmtFinType1_LwQ,BsmtFinType1_BLQ,GarageFinish_RFn,GarageFinish_Unf,GarageFinish_Fin,KitchenQual_Gd,KitchenQual_TA,KitchenQual_Fa,KitchenQual_Ex
0,856.0,854.0,3.0,706.0,150.0,548.0,2003,1710.0,8450.0,65.0,...,0,0,0,1,0,0,1,0,0,0
1,1262.0,0.0,3.0,978.0,284.0,460.0,1976,1262.0,9600.0,80.0,...,0,0,0,1,0,0,0,1,0,0
2,920.0,866.0,3.0,486.0,434.0,608.0,2001,1786.0,11250.0,68.0,...,0,0,0,1,0,0,1,0,0,0
3,796.0,566.0,1.0,732.0,64.0,480.0,1993,1362.0,14115.0,85.0,...,0,0,0,0,1,0,0,1,0,0
4,1694.0,0.0,3.0,1369.0,317.0,636.0,2004,1694.0,10084.0,75.0,...,0,0,0,1,0,0,1,0,0,0


*I use .corr() for spearman and pearson methods, and investigate the top 10 correlations*

In [13]:
corr_spearman = df_ohe.corr(method='spearman')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

OverallQual         0.815674
GrLivArea           0.712267
YearBuilt           0.663722
GarageArea          0.627355
GarageFinish_Unf   -0.622838
GarageYrBlt         0.621221
YearRemodAdd        0.617074
KitchenQual_TA     -0.602708
TotalBsmtSF         0.598211
1stFlrSF            0.556682
Name: SalePrice, dtype: float64

In [14]:
corr_pearson = df_ohe.corr(method='pearson')['SalePrice'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

OverallQual       0.794124
GrLivArea         0.705781
GarageArea        0.615975
TotalBsmtSF       0.598380
1stFlrSF          0.593201
KitchenQual_Ex    0.547833
KitchenQual_TA   -0.526552
YearBuilt         0.513021
YearRemodAdd      0.510237
MasVnrArea        0.504320
Name: SalePrice, dtype: float64

*I will use 5 variables with the higehst correlation for further investigation*

In [17]:
top_n = 5
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())

{'1stFlrSF',
 'GarageArea',
 'GarageFinish_Unf',
 'GrLivArea',
 'OverallQual',
 'TotalBsmtSF',
 'YearBuilt'}

I will investigate if

In [13]:
df_house_prices_records.dropna(inplace=True)
df_house_prices_records.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 1459
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   1stFlrSF      797 non-null    float64
 1   2ndFlrSF      797 non-null    float64
 2   BedroomAbvGr  797 non-null    float64
 3   BsmtExposure  797 non-null    object 
 4   BsmtFinSF1    797 non-null    float64
 5   BsmtFinType1  797 non-null    object 
 6   BsmtUnfSF     797 non-null    float64
 7   GarageArea    797 non-null    float64
 8   GarageFinish  797 non-null    object 
 9   GarageYrBlt   797 non-null    float64
 10  GrLivArea     797 non-null    float64
 11  KitchenQual   797 non-null    object 
 12  LotArea       797 non-null    float64
 13  LotFrontage   797 non-null    float64
 14  MasVnrArea    797 non-null    float64
 15  OpenPorchSF   797 non-null    float64
 16  OverallCond   797 non-null    int64  
 17  OverallQual   797 non-null    int64  
 18  TotalBsmtSF   797 non-null   

*Converting GarageYrBlt to int*

In [14]:
df_house_prices_records['GarageYrBlt'] = df_house_prices_records['GarageYrBlt'].astype(int)
df_house_prices_records.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 797 entries, 0 to 1459
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   1stFlrSF      797 non-null    float64
 1   2ndFlrSF      797 non-null    float64
 2   BedroomAbvGr  797 non-null    float64
 3   BsmtExposure  797 non-null    object 
 4   BsmtFinSF1    797 non-null    float64
 5   BsmtFinType1  797 non-null    object 
 6   BsmtUnfSF     797 non-null    float64
 7   GarageArea    797 non-null    float64
 8   GarageFinish  797 non-null    object 
 9   GarageYrBlt   797 non-null    int64  
 10  GrLivArea     797 non-null    float64
 11  KitchenQual   797 non-null    object 
 12  LotArea       797 non-null    float64
 13  LotFrontage   797 non-null    float64
 14  MasVnrArea    797 non-null    float64
 15  OpenPorchSF   797 non-null    float64
 16  OverallCond   797 non-null    int64  
 17  OverallQual   797 non-null    int64  
 18  TotalBsmtSF   797 non-null   

*Converting BsmtExposure to int*

In [15]:
df_house_prices_records['BsmtExposure'].unique()

array(['No', 'Gd', 'Mn', 'Av', 'None'], dtype=object)

In [16]:
df_house_prices_records['BsmtExposure'] = df_house_prices_records['BsmtExposure'].replace({"None":0, "No":1, "Mn":2, "Av":3, "Gd":4})

*Cheking BsmtExposure data type*

In [17]:
df_house_prices_records['BsmtExposure'].dtype

dtype('int64')

*Converting BsmtFinType1 to int*

In [18]:
df_house_prices_records['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'None', 'LwQ', 'BLQ'], dtype=object)

In [19]:
df_house_prices_records['BsmtFinType1'] = df_house_prices_records['BsmtFinType1'].replace({"None":0, "Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6})

*Cheking BsmtFinType1 data type*

In [20]:
df_house_prices_records['BsmtFinType1'].dtype

dtype('int64')

*Converting GarageFinish to int*

In [21]:
df_house_prices_records['GarageFinish'].unique()

array(['RFn', 'Unf', 'Fin'], dtype=object)

In [22]:
df_house_prices_records['GarageFinish'] = df_house_prices_records['GarageFinish'].replace({"None":0, "Unf":1, "RFn":2, "Fin":3})

*Cheking GarageFinish data type*

In [23]:
df_house_prices_records['GarageFinish'].dtype

dtype('int64')

*Converting KitchenQual to int*

In [24]:
df_house_prices_records['KitchenQual'].unique()

array(['Gd', 'TA', 'Fa', 'Ex'], dtype=object)

In [25]:
df_house_prices_records['KitchenQual'] = df_house_prices_records['KitchenQual'].replace({"Po":0, "Fa":1, "TA":2, "Gd":3, "Ex":4})

*Cheking KitchenQual data type*

In [26]:
df_house_prices_records['KitchenQual'].dtype

dtype('int64')

In [27]:
df_house_prices_records.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856.0,854.0,3.0,1,706.0,6,150.0,548.0,2,2003,...,8450.0,65.0,196.0,61.0,5,7,856.0,2003,2003,208500
1,1262.0,0.0,3.0,4,978.0,5,284.0,460.0,2,1976,...,9600.0,80.0,0.0,0.0,8,6,1262.0,1976,1976,181500
2,920.0,866.0,3.0,2,486.0,6,434.0,608.0,2,2001,...,11250.0,68.0,162.0,42.0,5,7,920.0,2001,2002,223500
5,796.0,566.0,1.0,1,732.0,6,64.0,480.0,1,1993,...,14115.0,85.0,0.0,30.0,5,5,796.0,1993,1995,143000
6,1694.0,0.0,3.0,3,1369.0,6,317.0,636.0,2,2004,...,10084.0,75.0,186.0,57.0,5,8,1686.0,2004,2005,307000


### Inherited houses

In [28]:
df_inherited_houses = pd.read_csv(f"inputs/heritage-housing-issues/house-price-20211124T154130Z-001/house-price/inherited_houses.csv")
df_inherited_houses.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,0,730.0,Unf,...,11622,80.0,0.0,0,6,5,882.0,140,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,0,312.0,Unf,...,14267,81.0,108.0,36,6,6,1329.0,393,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,0,482.0,Fin,...,13830,74.0,0.0,34,5,5,928.0,212,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,0,470.0,Fin,...,9978,78.0,20.0,36,6,6,926.0,360,1998,1998


#### DataFrame Summary

In [29]:
df_inherited_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       4 non-null      int64  
 1   2ndFlrSF       4 non-null      int64  
 2   BedroomAbvGr   4 non-null      int64  
 3   BsmtExposure   4 non-null      object 
 4   BsmtFinSF1     4 non-null      float64
 5   BsmtFinType1   4 non-null      object 
 6   BsmtUnfSF      4 non-null      float64
 7   EnclosedPorch  4 non-null      int64  
 8   GarageArea     4 non-null      float64
 9   GarageFinish   4 non-null      object 
 10  GarageYrBlt    4 non-null      float64
 11  GrLivArea      4 non-null      int64  
 12  KitchenQual    4 non-null      object 
 13  LotArea        4 non-null      int64  
 14  LotFrontage    4 non-null      float64
 15  MasVnrArea     4 non-null      float64
 16  OpenPorchSF    4 non-null      int64  
 17  OverallCond    4 non-null      int64  
 18  OverallQual   

*Converting 1stFlrSF	, 2ndFlrSF, BedroomAbvGr, EnclosedPorch, GrLivArea, LotArea, OpenPorchSF, WoodDeckSF to float.*
*Converting GarageYrBlt to int.*

In [30]:
df_inherited_houses['1stFlrSF'] = df_inherited_houses['1stFlrSF'].astype(float)
df_inherited_houses['2ndFlrSF'] = df_inherited_houses['2ndFlrSF'].astype(float)
df_inherited_houses['BedroomAbvGr'] = df_inherited_houses['BedroomAbvGr'].astype(float)
df_inherited_houses['EnclosedPorch'] = df_inherited_houses['EnclosedPorch'].astype(float)
df_inherited_houses['GrLivArea'] = df_inherited_houses['GrLivArea'].astype(float)
df_inherited_houses['LotArea'] = df_inherited_houses['LotArea'].astype(float)
df_inherited_houses['OpenPorchSF'] = df_inherited_houses['OpenPorchSF'].astype(float)
df_inherited_houses['WoodDeckSF'] = df_inherited_houses['WoodDeckSF'].astype(float)
df_inherited_houses['GarageYrBlt'] = df_inherited_houses['GarageYrBlt'].astype(int)
df_inherited_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       4 non-null      float64
 1   2ndFlrSF       4 non-null      float64
 2   BedroomAbvGr   4 non-null      float64
 3   BsmtExposure   4 non-null      object 
 4   BsmtFinSF1     4 non-null      float64
 5   BsmtFinType1   4 non-null      object 
 6   BsmtUnfSF      4 non-null      float64
 7   EnclosedPorch  4 non-null      float64
 8   GarageArea     4 non-null      float64
 9   GarageFinish   4 non-null      object 
 10  GarageYrBlt    4 non-null      int64  
 11  GrLivArea      4 non-null      float64
 12  KitchenQual    4 non-null      object 
 13  LotArea        4 non-null      float64
 14  LotFrontage    4 non-null      float64
 15  MasVnrArea     4 non-null      float64
 16  OpenPorchSF    4 non-null      float64
 17  OverallCond    4 non-null      int64  
 18  OverallQual   

*Converting BsmtExposure to int*

In [31]:
df_inherited_houses['BsmtExposure'].unique()

array(['No'], dtype=object)

In [32]:
df_inherited_houses['BsmtExposure'] = df_inherited_houses['BsmtExposure'].replace({"None":0, "No":1, "Mn":2, "Av":3, "Gd":4})

*Cheking BsmtExposure data type*

In [33]:
df_inherited_houses['BsmtExposure'].dtype

dtype('int64')

*Converting BsmtFinType1 to int*

In [34]:
df_inherited_houses['BsmtFinType1'].unique()

array(['Rec', 'ALQ', 'GLQ'], dtype=object)

In [35]:
df_inherited_houses['BsmtFinType1'] = df_inherited_houses['BsmtFinType1'].replace({"None":0, "Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6})

*Cheking BsmtFinType1 data type*

In [36]:
df_inherited_houses['BsmtFinType1'].dtype

dtype('int64')

*Converting GarageFinish to int*

In [37]:
df_inherited_houses['GarageFinish'].unique()

array(['Unf', 'Fin'], dtype=object)

In [38]:
df_inherited_houses['GarageFinish'] = df_inherited_houses['GarageFinish'].replace({"None":0, "Unf":1, "RFn":2, "Fin":3})

*Cheking GarageFinish data type*

In [39]:
df_inherited_houses['GarageFinish'].dtype

dtype('int64')

*Converting KitchenQual to int*

In [40]:
df_inherited_houses['KitchenQual'].unique()

array(['TA', 'Gd'], dtype=object)

In [41]:
df_inherited_houses['KitchenQual'] = df_inherited_houses['KitchenQual'].replace({"Po":0, "Fa":1, "TA":2, "Gd":3, "Ex":4})

*Cheking KitchenQual data type*

In [42]:
df_inherited_houses['KitchenQual'].dtype

dtype('int64')

In [43]:
df_inherited_houses.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896.0,0.0,2.0,1,468.0,3,270.0,0.0,730.0,1,...,11622.0,80.0,0.0,0.0,6,5,882.0,140.0,1961,1961
1,1329.0,0.0,3.0,1,923.0,5,406.0,0.0,312.0,1,...,14267.0,81.0,108.0,36.0,6,6,1329.0,393.0,1958,1958
2,928.0,701.0,3.0,1,791.0,6,137.0,0.0,482.0,3,...,13830.0,74.0,0.0,34.0,5,5,928.0,212.0,1997,1998
3,926.0,678.0,3.0,1,602.0,6,324.0,0.0,470.0,3,...,9978.0,78.0,20.0,36.0,6,6,926.0,360.0,1998,1998


---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [44]:
import os
try:
  os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

df_house_prices_records.to_csv(f"outputs/datasets/collection/house_price_records_clean.csv", index=False)
df_inherited_houses.to_csv(f"outputs/datasets/collection/inherited_houses_clean.csv", index=False)
